In [ ]:
%matplotlib inline
import sys
sys.path.append("..") # Adds the module to path

# DeepTrack

This notebook gives an overview of the capabilities of DeepTrack and how to model optical systems using predifined classes.

### What is DeepTrack?

DeepTrack is fundamentally permits to create, train and execute particle tracking models.

## 1. Generate images

A central capability is to generate images, which can be used to train and validate the operation of particle tracking models. For example, these images may be like the output of optical systems employed in experiments. 

The main idea of DeepTrack is that any image can be viewed as a series of features. These features take an input image and update it according to some update rule. For example, a feature can add a particle, introduce some noise, or image something through an optical device.

### Features and properties

Features in DeepTrack are classes implementing the class `Feature`. The way a feature updates an image is governed by the values passed to the class constructor. These inputs are converted to [properties](../examples/properties_example.ipynb). For example, a property could be the position of a particle.

In [ ]:
from deeptrack.scatterers import PointParticle

particle = PointParticle(
    position=(0, 0),
    position_unit="pixel", # Defaults to meter
    intensity=1,
)

The above code creates a feature (a point particle). This feature will always add a point particle at x=0, y=0. For machine learning, it may be more useful to add a particle at a random position. Thsi can be done by passing a lambda function that returns a random pair of numbers.

In [ ]:
import numpy as np

particle = PointParticle(
    position=lambda: np.random.rand(2) * 64,
    position_unit="pixel",
    intensity=1
)

A point scatterer on its own does not make much sense. We also need to define the optical system it is viewed through. Optical devices are also features, which convolves the input image with a pupil function. Here we will use a fluorescence microscope. 

By calling the feature `optics` with the scatterer `particle`, we create a new feature which resolves images of the particle as seen through the fluorescence microscope.

In [ ]:
from deeptrack.optics import Fluorescence

optics = Fluorescence(
    NA=0.8,
    wavelength=680e-9,
    magnification=10,
    resolution=1e-6,
    output_region=(0, 0, 64, 64)
)

imaged_particle = optics(particle)

### Adding more features

Features can be combined (see [features_example](features_example.ipynb)) using overloaded operators (+, \*, \*\* or ()). Here exemplify the add operator (+) and the power operator (\*\*).

In [ ]:
import matplotlib.pyplot as plt
# To image a feature, we call optics with the feature


imaged_particle.update()
output_image = imaged_particle.resolve()

plt.imshow(output_image, cmap='gray')
plt.show()

plt.imshow(output_image[:, :, 0])
plt.show()

In [ ]:
# The + operator
particle_1 = PointParticle(
    position=lambda: np.random.rand(2) * 64,
    position_unit="pixel",
    intensity=1
)

particle_2 = PointParticle(
    position=lambda: np.random.rand(2) * 64,
    position_unit="pixel",
    intensity=1
)

# two_particles is a new feature that first resolves particle_1 and then particle_2, then images it
two_particles = optics(particle_1 + particle_2)

output_image = two_particles.resolve()

plt.imshow(output_image[:, :, 0], cmap='gray')
plt.show()

In [ ]:
# The ** operator

particle = PointParticle(
    position=lambda: np.random.rand(2) * 64,
    position_unit="pixel",
    intensity=1
)

# five_particles is a feature that resolves five deep copies of particle, then images it
five_particles = optics(particle**5)

output_image = five_particles.resolve()

plt.imshow(output_image[:, :, 0])
plt.show()

### Adding noise

To make the image more realistic, we can add some noise. Noise can be added before (most commonly) or after applying the optical device.

In [ ]:
from deeptrack.noises import Offset, Poisson

# Adds a constant value to the background
offset = Offset(offset=0.01)

# Introduce Poisson noise to the image
poisson_noise = Poisson(snr=100)

# noisy_particles resolves five particles, then adds a offset, images it, then introduces poisson noise
noisy_particles = optics(particle**5) + offset + poisson_noise

output_image = noisy_particles.resolve()

plt.imshow(output_image[:, :, 0], cmap='gray')
plt.show()

## 2. Retrieve information about the image

To train a supervised machine-learning model, labled images are needed. When a features is resolved, it automatically stores the properties of all features used to create the image. This allows us to extract information about the image in order to use them to train machine-learning models.

Here, we extract the position of all the particles and plot them as red crossed on the generated image.

In [ ]:
def get_positions(image):
    # All properties are stored in the `properties` field of the output.
    positions = [property_dict["position"] for property_dict in image.properties if "position" in property_dict]
    return np.array(positions)



noisy_particles.update()
output_image = noisy_particles.resolve()

positions = get_positions(output_image)

plt.imshow(output_image[:, :, 0], cmap='gray')
plt.scatter(positions[:, 1], positions[:, 0], c="r", marker="x")
plt.show()

## 3. Wrap features in generators

Generators are ways to continuously resolve new images, and are the prefered interface to machine learning models. The default generator is defined in the module `generators`. We can also optionally pass a label function that will be called on every image.

In [ ]:
from deeptrack.generators import Generator

generator = Generator().generate(noisy_particles, get_positions)

for _ in range(4):
    # Outputs shape (1, height, width, 1)
    next_image, positions = next(generator)
    plt.imshow(np.squeeze(next_image), cmap='gray')
    plt.scatter(positions[0, :, 1], positions[0, :, 0], c="r", marker="x")
    plt.show()

## Finally, training a model

In [ ]:
from deeptrack.models import convolutional
from deeptrack.math import NormalizeMinMax

# DEFINE LABEL FUNCTION
def get_position(image):
    for propertydict in image.properties:
        if "position" in propertydict:
            return propertydict["position"] / 64
        
# DEFINE MODEL
tracker = convolutional(input_shape=(64, 64, 1), number_of_outputs=2)


# DEFINE TRAINING SET
normalization = NormalizeMinMax(min=0, max=1)
training_set = optics(particle) + offset + poisson_noise + normalization

# DEFINE GENERATOR
generator = Generator().generate(training_set, get_position, batch_size=32)



In [ ]:
# TRAIN TRACKER
tracker.fit(generator, epochs=100, steps_per_epoch=10)

In [ ]:
batch, labels = next(generator)
predictions = tracker.predict(batch) * 64
for image, position, prediction in zip(batch, labels * 64, predictions):
    plt.gray()
    plt.imshow(image[:, :, 0], cmap='gray')
    print(np.max(batch))
    plt.scatter(position[1], position[0], c='g', marker='x')
    plt.scatter(prediction[1], prediction[0], marker='o', facecolors=None, edgecolors='b')
    plt.show()